In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import backend as K
from tensorflow.keras import Model

import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

import sys

In [3]:
#read in data from example_failure_data_sets and choose sheet SYS1
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR2", engine="openpyxl")

In [4]:
#make train and test splits.
#change split fraction to be the ratio of testing data ex, 20/100 for 80/20 train/test
split_fraction = 10/100

(x_train, y_train), (x_test, y_test) = (df["FT"][:len(df["FT"]) - int(len(df["FT"]) * split_fraction)], df["FN"][:len(df["FT"]) - int(len(df["FT"]) *split_fraction)]), (df["FT"][len(df["FT"]) - int(len(df["FT"]) * split_fraction):], df["FN"][len(df["FT"]) - int(len(df["FT"])* split_fraction):])



In [5]:
#model being customized. Change activation, number of layers, layer size. Check names of layers and make sure they 
#are being added correctly in "call". x=layer(x) is the correct way. Always normalize. NN don't like unnormalized

b_values = list()
class MyModel(Model):
    def __init__(self, nodes = [60], activations= ["elu"], data = None):
        super(MyModel, self).__init__()
        self.b_flag = False
        self.a = 1
        self.d_in = preprocessing.Normalization()
        self.d_in.adapt(data)
        self.d = []
        for i in range(len(nodes)):
            if activations[i] ==  "x":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.x_activation))

            elif activations[i] ==  "e":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.e_activation))

            elif activations[i] == "1":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.minus_activation))

            elif activations[i] == "go":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.go_activation))
                self.b_flag = True
            else:
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=activations[i]))
            
        self.d_out = tf.keras.layers.Dense(1,activation=self.x_activation)
        if self.b_flag:
            self.b = tf.Variable(0.01)
    
    def e_activation(self, x):
        return K.exp(x)
    
    def minus_activation(self, x):
        return 1-K.switch(x>0, x, x)
    
    def x_activation(self, x):
        return K.switch(x>0, x, x)

    def go_activation(self, x):
        return self.a * (1-K.exp(-self.b * x) )
    
    def weibull_activation(self,x):
        return self.a *(1 - K.exp(-self.b * K.pow(x,self.c)))

    def dss_activation(self, x): #delayed s shaped
        return self.a * (1 - K.exp(-self.b*x) * (1 + self.b * x))

    def iss_activation(self, x): #inflection s shape
        return (self.a*(1-K.exp(-self.b*x)))/(1+self.c*K.exp(-self.b*x)) #ISS
        
    def call(self, x, **kwargs):
        if self.b_flag:
            b_values.append(self.b.numpy())
        x = self.d_in(x)
        for i in range(len(self.d)):
            x = self.d[i](x)
        x = self.d_out(x)
        return x

In [6]:
class NN_Bagging():
    def __init__(self, num_models = 10, lr = 0.1, activations = ["elu"], num_nodes = [50], min_delta=0.0001, patience=10, data = None, loss = "mean_squared_error" ):
        self.models = [MyModel(num_nodes, activations, data)]*num_models
        self.num_models = num_models
        self.lr = lr
        self.delta = min_delta
        self.patience = patience
        for i in range(0,self.num_models):
            self.models[i].compile(loss=loss, optimizer=tf.keras.optimizers.Adam(self.lr), run_eagerly=True)


    def fit(self, x, y):
        for i in range(0,self.num_models):
            self.models[i].fit(x, y, epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
    
    def train_on_batch(self, x, y, loss = "mean_squared_error"):
        for i in range(0,self.num_models):
            self.models[i].train_on_batch(x, y)

        
    def predict(self, x):
        y_res = [0]*len(x)
        y = None
        for m in self.models:
            y = m.predict(x) #y is what the model predicts
            for k in range(0,len(y)):
                y_res[k] += y[k][0]/self.num_models
        return y_res

In [18]:
## 80/20 split for different data sets


# df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="S2", engine="openpyxl")


# activations = ["elu", "relu", "go", "sigmoid", "tanh"]

# split_fraction= 20/100
# y_result= pd.DataFrame()

# t_start = time.time()
# for a in activations:
# #     for j in range(int(len(df) - 0.2 * len(df)), len(df)):  
#     #     (x_train, y_train), (x_test, y_test) = (df2["FT"][:j], df2["FN"][:j]), (df2["FT"][j:], df2["FN"][j:])
#     (x_train, y_train), (x_test, y_test) = (df["FT"][:len(df["FT"]) - int(len(df["FT"]) * split_fraction)], df["FN"][:len(df["FT"]) - int(len(df["FT"]) *split_fraction)]), (df["FT"][len(df["FT"]) - int(len(df["FT"]) * split_fraction):], df["FN"][len(df["FT"]) - int(len(df["FT"])* split_fraction):])
#     #     print(x_train)
#     model = NN_Bagging(data=x_train, min_delta=0.0001,lr=0.1, num_nodes=[60], activations=[a])
#     model.fit(x_train, y_train)
#     y_result["S2_{}".format(a)] = pd.Series(model.predict(df["FT"]))


# t_end = time.time()

# y_result.to_csv("./results/S2_80_20.csv")


In [17]:
# x_train, y_train = df2["FT"][int(len(df2["FT"] )* (split_fraction-10/100)):int(len(df2["FT"] )* split_fraction)], df2["FN"][int(len(df2["FT"] )* (split_fraction-10/100)):int(len(df2["FT"]) * split_fraction)]
# y_result = pd.DataFrame()


midpoints = []
for i in range(0, 10):
    midpoints.append(df2[ "FT"] [int( ( 0.5 + i ) / 10  * len(df2)) -1])
    print(int( ( 0.5 + i ) / 10  * len(df2)) -1)
    
print(midpoints)


5
18
31
44
57
69
82
95
108
121
[1863, 2940, 5357, 6872, 7972, 11899, 26054, 35692, 48208, 73414]


In [26]:
#testing block

fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]
y_results = {}
# fractions = [20, 40, 60, 80]
# fractions = [30, 60, 90]
# fractions = [50]
lr = 0.1
nodes = 60
activations = ["elu"]#, "relu", "go", "sigmoid", "tanh"]
t_start = time.time()

# model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=["tanh"])
# model.fit(df["FT"], df["FN"])

a = "elu"
datasets = [ ("CSR3", "CSR1"), ("CSR3", "CSR2"), ("CSR3", "S2"), ("S2", "CSR1"), ("S2", "CSR3"), ("SYS2", "CSR1"), ("SYS2", "CSR3")  ]

for d in datasets:
    df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[0], engine="openpyxl")
    df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[1], engine="openpyxl")
    midpoints_x = []
    midpoints_y = []
    for i in range(0, 10):
        midpoints_x.append(df2["FT"] [int( ( 0.5 + i ) / 10  * len(df2)) -1])
        midpoints_y.append(df2["FN"] [int( ( 0.5 + i ) / 10  * len(df2)) -1])
    
    y_results[d] = pd.DataFrame()
    model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=[a])
    model.fit(df["FT"], df["FN"])

    for j in range(len(fractions)):
        split_fraction = fractions[j]/100
#         model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=[a])
#         model.fit(df["FT"], df["FN"])
#         x_train, y_train = pd.Series(list(df2["FT"][:int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][:int(len(df2["FT"]) * split_fraction)]))
        x_train, y_train = pd.Series(list(df2["FT"][int(len(df2["FT"]) * (split_fraction - fractions[0]/100)):int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * (split_fraction -fractions[0]/100)):int(len(df2["FT"]) * split_fraction)]))
#         x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))]))
        model.fit(pd.Series(midpoints_x[:j] + list(x_train)), pd.Series(midpoints_y[:j] + list(y_train)) )
        y_results[d]["split:{}step:{}".format(fractions[j], fractions[0])] = pd.Series(model.predict(df2["FT"]))
t_end = time.time()

# #elu: 55 nodes
# #go: 60 nodes
with pd.ExcelWriter("./results/incremental_results_midpoints.xlsx") as ex:
    for i in y_results.keys():
        y_results[i].to_excel(ex, "{}, {}".format(i[0], i[1]), index=False)



In [36]:
print(t_end-t_start)
with pd.ExcelWriter("./results/transfer_results.xlsx") as ex:
    for i in y_results.keys():
        y_results[i].to_excel(ex, "{}, {}".format(i[0], i[1]), index=False)

872.4883849620819


In [6]:
#nodes/lr

fractions = 80
split_fraction = fractions/100

lr = [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75]
nodes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
y_result = pd.DataFrame()
t_start = time.time()

for l in range(len(lr)):
    for n in range(len(nodes)):
        x_train, y_train = pd.Series(list(df["FT"][:int(len(df["FT"]) * split_fraction)])), pd.Series(list(df["FN"][:int(len(df["FT"]) * split_fraction)]))
        x_test, y_test = pd.Series(list(df["FT"][int(len(df["FT"]) * split_fraction):])), pd.Series(list(df["FN"][int(len(df["FT"]) * split_fraction):]))

        model = NN_Bagging(lr=lr[l], num_nodes=[nodes[n]], data=x_train, min_delta=0.0001, activations=["elu"])
        model.fit(x_train, y_train)
        y_result["lr:{}_nodes:{}".format(lr[l], nodes[n])] = pd.Series(model.predict(df["FT"]))

t_end = time.time()

# # #elu: 55 nodes
# # #go: 60 nodes

y_result.to_excel("lr_node_csr3.xlsx", index=False)

In [16]:
#normal over multiple fractions

fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]


lr = 0.01
nodes = 55
y_result = pd.DataFrame()
models = []
t_start = time.time()

df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR1", engine="openpyxl")

for j in range(len(fractions)):
    split_fraction = (100-fractions[j])/100
#     (x_train, y_train), (x_test, y_test) = (df["FT"][:len(df["FT"]) - int(len(df["FT"]) * split_fraction)], df["FN"][:len(df["FT"]) - int(len(df["FT"]) *split_fraction)]), (df["FT"][len(df["FT"]) - int(len(df["FT"]) * split_fraction):], df["FN"][len(df["FT"]) - int(len(df["FT"])* split_fraction):])
    (x_train, y_train), (x_test, y_test) = (df2["FT"][:len(df2["FT"]) - int(len(df2["FT"]) * split_fraction)], df2["FN"][:len(df2["FT"]) - int(len(df2["FT"]) *split_fraction)]), (df2["FT"][len(df2["FT"]) - int(len(df2["FT"]) * split_fraction):], df2["FN"][len(df2["FT"]) - int(len(df2["FT"])* split_fraction):])
    model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=["relu"])
    model.fit(df["FT"], df["FN"])
    model.fit(x_train, y_train)
    y_result["split:{}".format(fractions[j])] = pd.Series(model.predict(df2["FT"]))

t_end = time.time()



In [17]:
y_result.to_excel("./results/csr3_csr2_incremental_2.xlsx", index=False)


In [9]:
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")

fractions = list(range( int(0.8 * len(df)), len(df)))



[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103]


,FN,IF,FT
0,1,33.0,33.0
1,2,9.0,42.0
2,3,4.0,46.0
3,4,66.0,112.0
4,5,0.5,112.5
...,...,...,...
99,100,3.0,14331.5
100,101,831.0,15162.5
101,102,43.0,15205.5
102,103,55.0,15260.5


In [25]:
#small steps
# dataset = "CSR3"
# df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=dataset, engine="openpyxl")
datasets = ["SYS1", "SYS2", "CSR3", "CSR2", "S2"]
activations = ["elu", "relu", "go", "sigmoid", "tanh"]
y_result = {}
t_start = time.time()

for d in datasets:
    df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d, engine="openpyxl")
    y_result[d] = pd.DataFrame()
    fractions = list(range( int(0.8 * len(df)), len(df) -1))
    print(d)
    for a in activations:
        for j in range(len(fractions)):
            split_fraction = fractions[j]
            (x_train, y_train), (x_test, y_test) = (df["FT"][:split_fraction], df["FN"][:split_fraction]), (df["FT"][split_fraction:], df["FN"][split_fraction:])
            print(split_fraction)
            model = NN_Bagging(data=x_train, min_delta=0.0001,lr=0.1, num_nodes=[60], activations=[a])
            model.fit(x_train, y_train)
            y_result[d]["{}_{}_{}".format(d, a, fractions[j])] = pd.Series(model.predict(df["FT"]))

t_end = time.time()

# # #elu: 55 nodes
# # #go: 60 nodes




SYS1
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
SYS2
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
CSR3
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
8

In [27]:
with pd.ExcelWriter("./results/small_steps.xlsx") as ex:
    for i in y_result.keys():
        y_result[i].to_excel(ex, "{}".format(i), index=False)

In [ ]:
# y_result3_layer = y_result
# y_result
# y_result.to_csv("results/tanh_steping_scratch.csv")

In [ ]:
# print("time: {}".format(t_end - t_start))

In [ ]:
# mod = MyModel(50)
# mod.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), run_eagerly=True)
# split_fraction = (20/100)

# mod.fit(x_train, y_train, epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=5), verbose=0)
# y = mod.predict(np.append(x_train,x_test))
# plt.plot(np.append(x_train,x_test), y)

In [ ]:
#predictive mean squared error. one of the more relavant statistics. Doesn't tell the whole story as size
#magnitude grows so the later points are going to have a higher penalty. Still error is high
pmse_list = []
for i in range(len(fractions)):    
    pmse =0
    split_fraction = fractions[i]/100
    y_test = pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))
#     print(y_test)
    count = 0
    for a, b in zip(y_result[y_result.keys()[i]][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))], y_test):
        if not np.isnan(a):
            count +=1
            pmse += (a-b)**2
#             print("{}, {}, {}".format( a, b,(a-b)**2))
    pmse = pmse/count
    pmse_list.append(pmse)



# for i in range(len(fractions)):    
#     pmse =0
#     split_fraction = (fractions[i]/100)
#     (x_train, y_train), (x_test, y_test) = (df2["FT"][:len(df2["FT"]) - int(len(df2["FT"]) * split_fraction)], df2["FN"][:len(df2["FT"]) - int(len(df2["FT"]) *split_fraction)]), (df2["FT"][len(df2["FT"]) - int(len(df2["FT"]) * split_fraction):], df2["FN"][len(df2["FT"]) - int(len(df2["FT"])* split_fraction):])
#     for a, b in zip(y_result[i][int(len(y_result[i])* (1-split_fraction)):], y_test):
#         pmse += (a-b)**2
#     #     print("{}, {}, {}".format( a, b,(a-b)**2))

#     print(pmse/len(y_test))
#     pmse_list.append(pmse/len(y_test))
# print(len(y_test))

In [ ]:
print(pmse_list)
# y_result[y_result.keys()[i]]
# pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]))]))
# a = 8
# split_fraction = fractions[a]/100
# y_test = pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))


# [x for x in zip(y_result[y_result.keys()[a]][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))], y_test)]



In [ ]:
mse_list = []
for i in range(len(lr)* len(num_nodes)):    
    mse =0;
    for a, b in zip(y_result[y_result.keys()[i]][int(0.8* len(y_result[y_result.keys()[i]])):], list(y_test)):
#         print(a,b)
        mse += (a-b)**2
    #     print("{}, {}, {}".format( a, b,(a-b)**2))

    print(mse/len(y_train))
    mse_list.append(mse/len(y_train))


In [ ]:
i = -1
split_fraction = fractions[i]/100
x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))


y_plot = plt.plot(x_test[:-1], y_result[y_result.keys()[i]] , "dimgray", linestyle="--", marker="o", label="predicted")
actual_plot = plt.plot(x_test,  y_test, "blue", linestyle="--", marker="o", label="actual")

# y_result[y_result.keys()[-10]]
# actual = plt.plot(df["FT"], df["FN"], "black", label="sys2")
plt.xlabel("Time")
plt.ylabel("Number of Failures")
plt.legend()
#     plt.savefig("./results/{}_testing_fraction_go.jpg".format(fractions[x]),dpi=300, bbox_inches = "tight" )
plt.show()

In [ ]:
# #["{0}/{1}".format(136-x, x) for x in fractions]
# pmse_plot = plt.plot(lr,  pmse_list, "dimgray", linestyle="--", marker="o", label="ELU PMSE")
# mse_plot = plt.plot(lr, mse_list, "silver", marker="o",linestyle="-.", label = "ELU MSE")
# plt.xlabel("Learning Rates")
# plt.ylabel("MSE")
# # plt.gca().invert_xaxis()
# plt.legend()
# plt.savefig("./results/elu_pmse_mse_lr.jpg",dpi=300, bbox_inches = "tight" )
# plt.show()

In [ ]:
# one = plt.plot(num_nodes, pmse_list, "red", linestyle="--", marker="o", label="1 layers PMSE")
# two = plt.plot(num_nodes, pmse_list2, "black", marker="o",linestyle="-.", label = "2 layers PMSE")
# three = plt.plot(num_nodes, pmse_list3, "blue", marker="o",linestyle="-.", label = "3 layers PMSE")


# one1 = plt.plot(num_nodes, mse_list, "red", linestyle="None", marker="x", label="1 layers MSE")
# two2 = plt.plot(num_nodes, mse_list2, "black", marker="x",linestyle="None", label = "2 layers MSE")
# three3 = plt.plot(num_nodes, mse_list3, "blue", marker="x",linestyle="None", label = "3 layers MSE")

# # actual = plt.plot(df2["FT"], df2["FN"], "black", label="sys1")
# plt.xlabel("Number of Nodes")
# plt.ylabel("PMSE")
# plt.legend()
# # plt.savefig("results/elu_layers_80_20.jpg",dpi=300, bbox_inches = "tight" )
# plt.show()

In [ ]:
incremental_pmse = pmse_list
# incremental_mse = mse_list
incremental_results = y_result

# scratch_pmse = pmse_list
# scratch = y_result
# scratch_mse = mse_list

In [ ]:
# for x in range(len(incremental_results)):
    
#     incremental = plt.plot(df2["FT"],  incremental_results[x], "dimgray", linestyle="--", marker="o", label="incremental")
# #     pl = plt.plot(df2["FT"], scratch[x], "silver", marker="o",linestyle="-.", label = "elu")
#     actual = plt.plot(df2["FT"], df2["FN"], "black", label="sys2")
#     plt.xlabel("Time")
#     plt.ylabel("Number of Failures")
#     plt.legend()
# #     plt.savefig("./results/{}_testing_fraction_go.jpg".format(fractions[x]),dpi=300, bbox_inches = "tight" )
#     plt.show()

In [ ]:
a = 5
for a in range(0,9):
    incremental = plt.plot(df["FT"][int(len(df["FT"])*(fractions[a]/100 - fractions[0]/100) ):int(len(df["FT"])*(fractions[a]/100 + fractions[0]/100) )],  y_result[incremental_results.keys()[a]][int(len(df["FT"])*(fractions[a]/100 - fractions[0]/100) ):int(len(df["FT"])*(fractions[a]/100 + fractions[0]/100) )], "dimgray", linestyle="--", marker="o", label="elu incremental")
    # pl = plt.plot(fractions[:], scratch_pmse[:], "silver", marker="o",linestyle="-.", label = "elu scratch")
    actual = plt.plot(df["FT"][int(len(df["FT"])*(fractions[a]/100 - fractions[0]/100) ):int(len(df["FT"])*(fractions[a]/100 + fractions[0]/100) )], df["FN"][int(len(df["FT"])*(fractions[a]/100 - fractions[0]/100) ):int(len(df["FT"])*(fractions[a]/100 + fractions[0]/100) )], "black", label="sys1")
    plt.xlabel("Train Fraction")
    plt.ylabel("PMSE")

#     plt.axvline(x=df["FT"][int(len(df["FT"])*(fractions[a]/100 - fractions[0]/100) )], color="blue",linestyle="-")

    plt.axvline(x=df["FT"][int(len(df["FT"])*fractions[a]/100)], color="black",linestyle="-")
#     if a < 8:
#         plt.axvline(x=df["FT"][int(len(df["FT"])*(fractions[a]/100 + fractions[0]/100) )], color="black",linestyle="--")
#     else:
#         plt.axvline(x=df["FT"][len(df["FT"]) - 1], color="black",linestyle="--")    
    plt.legend()
    # plt.savefig("results/40_predict_elu_pmse.jpg",dpi=300, bbox_inches = "tight" )
    plt.show()